In [332]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.colors
import statsmodels.formula.api as smf
import seaborn as sns
from cem.match import match
from cem.coarsen import coarsen
from cem.imbalance import L1
import statsmodels.api as sm


In [333]:
matplotlib.rc('font', family='Arial') 

In [334]:

pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df['Country Code']=pop_df['Country Code'].apply(lambda x: x.lower())
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values

excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS",   # Timor-Leste
    "GEO", #Georgia
    'SWZ', 
    'PRK', #North Korea
]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]


possible_iso=list(set(possible_countries)-set(excluded_iso3_codes))

In [397]:

df = pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df=df[df['country'].isin(possible_iso)]
df.rename(columns={'aggregated_value': 'count', 'country': 'Mention_country', 'affiliation_country': 'Aff_country'}, inplace=True)
df = df[df['year'].isin(np.arange(2002, 2020))]
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=['MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
df=df[~df['subjarea'].isin(physical_sciences)]

df=df.groupby(['year', 'Mention_country'])['count'].sum().reset_index()


In [398]:
data=pd.read_csv(r"C:\Users\Yasaman\Downloads\scopus_2024_V1_scholarlymigration_country_enriched.csv")
data=data[data['year'].isin(np.arange(2002, 2020))]
data=data[['iso3code', 'incomelevel', 'gdp_per_capita', 'year', 'population', 'region']].dropna()
data.rename(columns={'iso3code':'Mention_country'}, inplace=True)
data['Mention_country']=data['Mention_country'].apply(lambda x: x.lower())
df=df.merge(data, on=['Mention_country', 'year'], how='outer')

In [413]:
countries_to_remove=[]
for c  in df['Mention_country'].unique():
    if ((~df['count'].isna()) & (df['Mention_country'] == c)).sum()<=15:
        countries_to_remove.append(c)

print(len(countries_to_remove))

64


In [415]:
df = df[~df['Mention_country'].isin(countries_to_remove)].reset_index(drop=True)


In [419]:
# Define the required year range
required_years = list(range(2002, 2020))

# Get the unique countries
unique_countries = df["Mention_country"].unique()

# Create a complete DataFrame with all country-year combinations
full_data = []
for country in unique_countries:
    country_data = df[df["Mention_country"] == country]
    existing_years = set(country_data["year"])
    
    for year in required_years:
        if year in existing_years:
            row = country_data[country_data["year"] == year].iloc[0].to_dict()
        else:
            row = {
                "year": year,
                "Mention_country": country,
                "count": 0,
                "incomelevel": country_data["incomelevel"].iloc[0] if not country_data.empty else np.nan,
                "gdp_per_capita": np.nan,
                "population": np.nan,
                "region": country_data["region"].iloc[0] if not country_data.empty else np.nan,
            }
        full_data.append(row)

# Convert to DataFrame
df_complete = pd.DataFrame(full_data)


In [434]:
df_complete['treated']=df_complete['Mention_country'].isin(abbr).astype(int)
df_complete['treated_CW']=df_complete['Mention_country'].isin(['yem', 'lby', 'syr']).astype(int)
df_complete['treated_GO']=df_complete['Mention_country'].isin(['egy', 'tun']).astype(int)
df_complete['treated_GC']=df_complete['Mention_country'].isin(['omn', 'kwt', 'bhr', 'mar','jor']).astype(int)
df_complete['post']=df_complete['year'].apply(lambda x: 0 if x>=2002 and x<=2010 else 1 )
df_complete['count']=df_complete['count'].fillna(0)
df_complete['log_count']=np.log(df_complete['count']+1)
df_complete[['incomelevel', 'region', 'gdp_per_capita', 'population']] = df_complete.groupby('Mention_country')[['incomelevel', 'region', 'gdp_per_capita', 'population']].ffill()
df_complete[['incomelevel', 'region', 'gdp_per_capita', 'population']] = df_complete.groupby('Mention_country')[['incomelevel', 'region', 'gdp_per_capita', 'population']].bfill()
df_complete['log_gdp']=np.log(df_complete['gdp_per_capita'])
df_complete['log_population']=np.log(df_complete['population'])
df_complete[df_complete['Mention_country'].isin(possible_iso)].reset_index(drop=True)

,year,Mention_country,count,incomelevel,gdp_per_capita,population,region,treated,treated_CW,treated_GO,treated_GC,post,log_count,log_gdp,log_population,treated:post,log_gdp_scaled
0,2002.0,afg,53.925109,LIC,182.174037,21000256.0,South Asia,0,0,0,0,0,4.005971,5.204962,16.860045,0,-2.028242
1,2003.0,afg,62.787674,LIC,199.643228,22645130.0,South Asia,0,0,0,0,0,4.155560,5.296532,16.935455,0,-1.969757
2,2004.0,afg,72.018290,LIC,221.830531,23553551.0,South Asia,0,0,0,0,0,4.290710,5.401914,16.974787,0,-1.902449
3,2005.0,afg,79.680195,LIC,254.115274,24411191.0,South Asia,0,0,0,0,0,4.390493,5.537788,17.010552,0,-1.815666
4,2006.0,afg,84.109066,LIC,274.015394,25442944.0,South Asia,0,0,0,0,0,4.443934,5.613184,17.051949,0,-1.767511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,2015.0,zwe,416.835292,LMC,1410.329173,14154937.0,Sub-Saharan Africa,0,0,0,0,1,6.035087,7.251578,16.465574,0,-0.721068
2606,2016.0,zwe,474.749376,LMC,1421.787791,14452704.0,Sub-Saharan Africa,0,0,0,0,1,6.164891,7.259670,16.486392,0,-0.715899
2607,2017.0,zwe,526.176430,LMC,1192.107012,14751101.0,Sub-Saharan Africa,0,0,0,0,1,6.267535,7.083478,16.506828,0,-0.828434
2608,2018.0,zwe,599.204753,LMC,2269.177012,15052184.0,Sub-Saharan Africa,0,0,0,0,1,6.397271,7.727172,16.527034,0,-0.417306


In [424]:
df_complete.to_csv(r'C:\Users\Yasaman\Downloads\check_DiD.csv')

In [435]:
# Running the Difference-in-Differences regression
model = smf.ols("log_count ~ treated * post + log_gdp  + C(Mention_country) + C(year)", data=df_complete).fit()
# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_count   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     848.9
Date:                Mon, 10 Mar 2025   Prob (F-statistic):               0.00
Time:                        11:58:13   Log-Likelihood:                 291.36
No. Observations:                2610   AIC:                            -254.7
Df Residuals:                    2446   BIC:                             707.5
Df Model:                         163                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [437]:
# Running the Difference-in-Differences regression
model = smf.ols("log_count ~ treated_CW * post+treated_GC * post+treated_GO* post +log_gdp+ C(Mention_country) + C(year)", data=df_complete).fit()
# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_count   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                     855.5
Date:                Mon, 10 Mar 2025   Prob (F-statistic):               0.00
Time:                        12:01:59   Log-Likelihood:                 317.99
No. Observations:                2610   AIC:                            -304.0
Df Residuals:                    2444   BIC:                             670.0
Df Model:                         165                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [430]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
df_complete['treated:post']=df_complete['treated']*df_complete['post']
# Select the independent variables (excluding categorical variables handled by C())
independent_vars = df_complete[['treated', 'post', 'treated:post', 'log_gdp', 'log_population']]

# Compute VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = independent_vars.columns
vif_data["VIF"] = [variance_inflation_factor(independent_vars.values, i) for i in range(len(independent_vars.columns))]

print(vif_data)


         Variable        VIF
0         treated   2.153278
1            post   2.210859
2    treated:post   2.150505
3         log_gdp  23.878211
4  log_population  23.153844
